# Combine Sub-SBML models with TXTL Machinery
---

Do this so you can incorporate temperature control <br>
2020.08.14

In [1]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
#import pylab as plt 
#%matplotlib inline

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row

bokeh.io.output_notebook()

import colorcet

from subsbml import createNewSubsystem, createSubsystem

Loading BokehJS ...

In [2]:
palette = colorcet.b_glasbey_category10
palette.remove( '#d62628')

In [3]:
p_atp = bokeh.plotting.figure(width = 450, height = 350,
                             title = 'ATP Compare',
                             x_axis_label = 'time (sec)',
                             y_axis_label = 'concentration (nM)')

## Combine atp synthase with txtl machinery 
---

Load in subsystem models

In [4]:
# atp synthase models
ss1_atpsyn_txtl = createSubsystem('atpsynthase_machinery_txtl.xml', subsystemName = 'atpsyn_txtl')
ss2_atpsyn_mi = createSubsystem('atpsynthase_membrane_integration.xml', subsystemName = 'atpsyn_mi')
ss3_atpsynthesis = createSubsystem('atp_synthesis_through_atpsynthase_try.xml', subsystemName = 'atpsynthesis')
ss4_atp_use = createSubsystem('atp_50_use.xml', subsystemName = 'atp_use')

Combine all models

In [25]:
atp_subsystem_w_use = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use

Indicate initial conditions and timepoints, simulate with bioscrape, and get species IDs that you may want to plot

In [31]:
# SIMULATE WITH ATP SYNTHASE
# Initial conditions
spec_list_10 = ['dna_atpsyn', 'proton_outside_H',
                 'protein_RNAP_machinery', 
                'protein_Ribo_machinery', 'protein_RNAse_machinery']
for i,spec in enumerate(spec_list_10):
    atp_subsystem_w_use.setSpeciesAmount(spec,10)
    
atp_subsystem_w_use.setSpeciesAmount('metabolite_atp',1500000)


# timepoints and get WITH ATP SYNTHASE
timepoints = np.linspace(0,31600,100) #31600
results1, _ = atp_subsystem_w_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_atp').getId()
ADP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_adp').getId()

outside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_outside_H').getId()
inside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_inside_H').getId()
atp_synthase_b_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn').getId()

odeint failed with mxstep=500...

Perform simulations with the system with **only** ATP use, no atp synthase

In [32]:
# SIMULATE WITHOUT ATP SYNTHASE
#ss4_atp_use.setSpeciesAmount('proton_outside_H',10)
ss4_atp_use.setSpeciesAmount('metabolite_atp',10)


results2, _ = ss4_atp_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

# GET ATP species ID s
ATP_id2 = ss4_atp_use.getSpeciesByName('metabolite_atp').getId()

Plot!

In [33]:
# PLOT WITH ATP SYNTHASE

plt_timepoints = timepoints/60/60

p_w_atp = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Synthase',
                          x_axis_label = 'time (hrs)',
                          y_axis_label = 'concentration (nM)')
plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_w_atp.line(plt_timepoints, results1[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    

# PLOT WITHOUT ATP SYNTHASE
p_no_atp = bokeh.plotting.figure(width = 450, height = 350, title = 'only ATP Use',
                          x_axis_label = 'time (hrs)',
                          y_axis_label = 'concentration (nM)')
p_no_atp.line(plt_timepoints, results2[ATP_id2], line_width = 2, legend_label = 'atp', color = palette[0])

p_compare = bokeh.plotting.figure(width = 450, height = 350, title = 'Compare ATP ',
                          x_axis_label = 'time (hr)',
                          y_axis_label = 'concentration (nM)')
p_compare.line(plt_timepoints, results1[ATP_id], legend_label = 'ATP Synthase, ATP Use', line_width = 2, color = palette[0])
p_compare.line(plt_timepoints, results2[ATP_id2], legend_label = 'ATP Use', line_width = 2, color = palette[1])


p_w_atp.legend.location = 'bottom_right'
p_w_atp.legend.click_policy = 'hide'
    
bokeh.io.show(row(p_w_atp, p_no_atp))
bokeh.io.show(p_compare)

# p_atp.line(plt_timepoints, results1[ATP_id], line_width = 2, color = palette[1], legend_label = 'with ATP Synthase, ATP Use')
# p_atp.line(plt_timepoints, results2[ATP_id2],line_width = 2, color = palette[2],legend_label = 'ATP Use only')

#bokeh.io.show(p_atp)


Why does bound atp synthase shoot infinitely high:( got em, it's because you kept remaking the transcript silly

## Combine all proton pump with txtl machinery
---

In [9]:
#ss5_pp_txtl = createSubsystem('proton_pump_tx_tl.xml', subsystemName = 'pp_txtl')
ss5_pp_txtl = createSubsystem('proton_pump_machinery_txtl.xml', subsystemName = 'pp_txtl')
ss6_pp_mi = createSubsystem('proton_pump_membrane_integration.xml', subsystemName = 'pp_mi')
ss7_protontransport = createSubsystem('proton_transport_through_proton_pump.xml', subsystemName = 'protontransport')

In [10]:
pp_subsystem = ss5_pp_txtl + ss6_pp_mi + ss7_protontransport

In [11]:
# Initial conditions

spec_list_10 = ['dna_proton_pump', 'proton_outside_H', 'protein_RNAP_machinery', 
                'protein_Ribo_machinery', 'protein_RNAse_machinery']
for i,spec in enumerate(spec_list_10):
    pp_subsystem.setSpeciesAmount(spec,10)
pp_subsystem.setSpeciesAmount('metabolite_atp',1500000)

# timepoints and get results
timepoints = np.linspace(0,31600,100)
results, _ = pp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = pp_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = pp_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = pp_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = pp_subsystem.getSpeciesByName('proton_inside_H').getId()
proton_pump_b_protein_id = pp_subsystem.getSpeciesByName('protein_proton_pump_b').getId()
proton_pump_f_protein_id = pp_subsystem.getSpeciesByName('protein_proton_pump').getId()



dna_pp = pp_subsystem.getSpeciesByName('dna_proton_pump').getId()

odeint failed with mxstep=500...

In [12]:
plt_timepoints = timepoints/60/60

p_pp = bokeh.plotting.figure(width = 450, height = 350, title = 'with Proton Pump',
                            x_axis_label = 'time (hrs)',
                            y_axis_label = 'concentration (nM)')
plot_list = [ATP_id, outside_H_id, inside_H_id, proton_pump_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound proton pump']
for i,item in enumerate(plot_list):
    p_pp.line(plt_timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
p_pp.legend.click_policy = 'hide'
p_pp.legend.location = 'bottom_right'
bokeh.io.show(p_pp)

## Combine atp synthase and proton pump subsystems (with machinery txtl)
---

In [13]:
entire_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use + ss5_pp_txtl + ss6_pp_mi + ss7_protontransport

In [14]:
# Initial conditions
spec_list_10 = ['dna_proton_pump', 'dna_atpsyn','proton_outside_H','protein_RNAP_machinery', 
                'protein_Ribo_machinery', 'protein_RNAse_machinery']
for i,spec in enumerate(spec_list_10):
    entire_subsystem.setSpeciesAmount(spec,10)
entire_subsystem.setSpeciesAmount('metabolite_atp',1500000)

# timepoints and get results
# timepoints = np.linspace(0,10,100)
# results, _ = entire_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id3 = entire_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = entire_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = entire_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = entire_subsystem.getSpeciesByName('proton_inside_H').getId()

proton_pump_b_protein_id = entire_subsystem.getSpeciesByName('protein_proton_pump_b').getId()
proton_pump_f_protein_id = entire_subsystem.getSpeciesByName('protein_proton_pump').getId()

atp_synthase_b_protein_id = entire_subsystem.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = entire_subsystem.getSpeciesByName('protein_atpsyn').getId()


In [15]:
timepoints = np.linspace(0,31600,100)
results_everything, _ = entire_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

odeint failed with mxstep=500...

In [16]:
from bokeh.models.annotations import Title



In [17]:
plt_timepoints = timepoints/60/60

p3 = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Synthase, Proton Pump, ATP Use',
                          x_axis_label = 'time (hrs)',
                          y_axis_label = 'concentration (nM)')
plot_list = [ATP_id3, outside_H_id, inside_H_id, atp_synthase_b_protein_id, proton_pump_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'bound proton pump']
for i,item in enumerate(plot_list):
    p3.line(plt_timepoints, results_everything[item], line_width = 2, legend_label = plot_str[i], color = palette[i])

p_compare.line(plt_timepoints, results_everything[ATP_id3], line_width = 2, legend_label = 'ATP Synthase, Proton Pump, ATP Use', color = palette[3])
#p3.line(plt_timepoints, results_everything[ATP_id3])
p3.legend.click_policy = 'hide'
p3.legend.location = 'center_right'
#p_compare.title = 'Compare ATP All'
t = Title()
t.text = 'Compare ATP All'
p_compare.title = t
p_compare.legend.location = 'center_right'
bokeh.io.show(p3)
bokeh.io.show(p_compare)

## Compare with atpsyn + proton pump, only atpsyn, only atpuse (with txtl machinery)
---

In [18]:
bokeh.io.show(row(p3,p_w_atp, p_no_atp))

Cool. What's going on with the second plot. How come it looks like atp isn't being consumed?? Fix those params?? Next do temperature sensitivity.

In [19]:
p_atp = bokeh.plotting.figure(width = 450, height = 350,
                             title = 'ATP Compare',
                             x_axis_label = 'time (sec)',
                             y_axis_label = 'concentration (nM)')

p_atp.line(timepoints, results_everything[ATP_id3],line_width = 2, color = palette[0],legend_label = 'with ATP Synthase, Proton Pump, ATP Use')
p_atp.line(timepoints, results[ATP_id], line_width = 2, color = palette[1], legend_label = 'with ATP Synthase, ATP Use')
p_atp.line(timepoints, results2[ATP_id2],line_width = 2, color = palette[2],legend_label = 'ATP Use only')
p_atp.legend.location = 'center_right'
bokeh.io.show(p_atp)

In [20]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,scipy,biocrnpyler,bioscrape,colorcet,subsbml
  

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.2.1
jupyterlab 1.2.6
scipy 1.4.1
biocrnpyler unknown
bioscrape 1.0.0
colorcet 2.0.2
subsbml unknown
